# 03_BT_01_Betas_v1

This notebook calculates the __first version__ of the betas (Value given from people in interviews to physical variables in the analysed territories)

## Import libraries

In [1]:
first_folder_path = '../'

In [3]:
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd
from geopandas.tools import overlay
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import seaborn as sns
import random
import scipy.stats as stats

import os
import sys
module_path = os.path.abspath(os.path.join(first_folder_path))
if module_path not in sys.path:
    sys.path.append(module_path)
    import src
else:
    import src

## Notebook config

## Input - Load surveys

In [8]:
# Read file (Converted to csv UTF-8)
surveys = pd.read_csv(first_folder_path + "data/vref/output/table/encuestas/Encuesta_caminata(Guadalajara)(1-553).csv")

# Show
surveys.head(2)

,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,...,3,4,5,6,7,8,9,10,11,12
0,1,6/28/24 12:24:50,6/28/24 12:25:46,anonymous,NaN,NaN,DS,6/25/2024,001,TA_Miramar,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
1,2,6/28/24 12:25:58,6/28/24 12:28:18,anonymous,NaN,NaN,DS,6/25/2024,002,TA_Miramar,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"


In [9]:
surveys.columns

Index(['ID', 'Hora de inicio', 'Hora de finalización', 'Correo electrónico',
       'Nombre', 'Hora de la última modificación', 'Encuestador',
       'Fecha de realización', 'Encuesta #', 'Territorio de análisis',
       'Lugar donde se realizó la encuesta', 'Género', 'Edad (años)',
       'Estrato socioeconómico (NO APLICA)', 'Ocupación principal',
       'Nivel educativo completado',
       '¿Poseen alguno de los siguientes vehículos en su vivienda?',
       'Indique su lugar de residencia (Municipio/comuna/barrio)',
       '¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?',
       '¿Cómo suple las necesidades de equipamientos educativos?',
       '¿Cómo suple las necesidades de equipamientos de cultura o deportivos?',
       '¿Cuáles?',
       '¿Hacia qué tipo de equipamiento/sitio de interés se dirige?',
       '¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)',
       'En una semana típica, ¿Cuántas veces 

## Filter for data of interest

In [10]:
# INFO - Columns that hold relation with physical variables.

# POPULATION DENSITY
# 'Column' --> 36. (A lo largo de su recorrido, ¿Sientes que hay personas a tu alrededor que pudieran apoyarte en caso de algún problema?)
# Sí --> Positivo (Hay densidad)
# No --> Negativo (No hay densidad)

# INTERSECTIONS DENSITY
# '2' --> 37. (A lo largo de su recorrido, ¿Sientes que hay diversas intersecciones que te permiten cruzar las calles de manera segura?)
# Sí --> Positivo (Hay densidad de intersecciones)
# No --> Negativo (Faltan intersecciones)

# '3' --> 38. (¿Se te hace largo y aburrido el recorrido?)
# Sí --> Negativo (Faltan intersecciones?)
# No --> Positivo (Hay intersecciones?)

# '4' --> 39. (A lo largo de tu recorrido, ¿Es posible desviarse en múltiples puntos/intersecciones para cambiar de ruta?)
# Sí --> Positivo (Hay densidad de intersecciones)
# No --> Negativo (Faltan intersecciones)

# LAND USE
# '5' --> 40. (A lo largo de tu recorrido, ¿Puedes aprovechar para acceder a múltiples servicios y satisfacer necesidades diversas (realizar vueltas/pendientes, comprar alimentos, divertirse, etc.)?)
# Sí --> Positivo (Hay varios usos)
# No --> Negativo (Faltan usos)

# '6' --> 41. (A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?)
# Sí --> Positivo (Hay varios usos)
# No --> Negativo (Faltan usos)

# '7' --> 42. (A lo largo de tu recorrido, ¿Encuentras elementos que hacen que tu caminata sea poco monótona?)
# Sí --> Positivo (Hay varios usos)
# No --> Negativo (Faltan usos)

# VEGETATION INDEX
# '8' --> 43. (A lo largo de tu recorrido, ¿Sientes que el tipo de vegetación te genera sombra y protección del sol?)
# Sí --> Positivo (Hay vegetación)
# No --> Negativo (Falta vegetación)

# '9' --> 44. (A lo largo de tu recorrido, ¿Sientes que la presencia y el tipo de vegetación crea un entorno agradable?)
# Sí --> Positivo (Hay vegetación)
# No --> Negativo (Falta vegetación)

# SLOPE
# '10' --> 45. (A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?)
# Sí --> Negativo (La pendiente afecta)
# No --> Positivo (La pendiente no afecta)

# SIDEWALK PRESENCE
# '11' --> 46. (A lo largo de tu recorrido, ¿Te sientes seguro(a) ante la presencia de vehículos?)
# Sí --> Positivo (La infraestructura te hace sentir seguro)
# No --> Negativo (La infraestructura no es suficiente para hacerte sentir seguro)

# '11' --> 47. (A lo largo de tu recorrido, ¿Sientes que la continuidad de andenes te permite caminar de manera cómoda (no hay: cambios de niveles, interrupciones repentinas del andén, presencia de obstáculos, cambios en el ancho del andén, etc.)?)
# Sí --> Positivo (La infraestructura te permite andar cómodamente)
# No --> Negativo (La infraestructura no es suficiente para permitirte andar cómodamente)

In [30]:
# Filter for data of interest
surveys_f = surveys[['ID','Territorio de análisis','Column', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']]

# Rename columns
surveys_f.rename(columns={'Territorio de análisis':'Territory',
                          'Column':'36_text',
                          '2':'37_text',
                          '3':'38_text',
                          '4':'39_text',
                          '5':'40_text',
                          '6':'41_text',
                          '7':'42_text',
                          '8':'43_text',
                          '9':'44_text',
                          '10':'45_text',
                          '11':'46_text',
                          '12':'47_text'
                         },inplace=True
                )

# Show
surveys_f.head(2)

,ID,Territory,36_text,37_text,38_text,39_text,40_text,41_text,42_text,43_text,44_text,45_text,46_text,47_text
0,1,TA_Miramar,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
1,2,TA_Miramar,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"


## Assign values to survey's answers

In [31]:
# Mapping dictionary
mapping = {"Totalmente en desacuerdo": -2,
           "En desacuerdo": -1,
           "Ni de acuerdo, ni en desacuerdo": 0,
           "De acuerdo": 1,
           "Totalmente de acuerdo": 2
}

# Apply mapping
question_numbers = ['36','37','38','39','40','41','42','43','44','45','46','47']
for question in question_numbers:
    surveys_f[question+'_value'] = surveys_f[question+'_text'].map(mapping)

# Reorder columns to visualize
ordered_cols = ['ID','Territory']
for question in question_numbers:
    ordered_cols.append(question+'_text')
    ordered_cols.append(question+'_value')
surveys_f = surveys_f[ordered_cols]

# Show
surveys_f.head(5)

,ID,Territory,36_text,36_value,37_text,37_value,38_text,38_value,39_text,39_value,...,43_text,43_value,44_text,44_value,45_text,45_value,46_text,46_value,47_text,47_value
0,1,TA_Miramar,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,...,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0
1,2,TA_Miramar,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,...,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0
2,3,TA_Miramar,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,...,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0
3,4,TA_Miramar,Totalmente de acuerdo,2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2,Totalmente de acuerdo,2,...,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2,Totalmente de acuerdo,2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2
4,5,TA_Miramar,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,...,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0,"Ni de acuerdo, ni en desacuerdo",0
